In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (AddContactMaterial, AddMultibodyPlant, BasicVector,
                         CoulombFriction, DeformableBodyConfig, DiagramBuilder,
                         Filament, FixedOffsetFrame, GeometryInstance,
                         IllustrationProperties, LeafSystem, Meshcat,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         ModelInstanceIndex, MultibodyPlantConfig,
                         PdControllerGains, PrismaticJoint, ProximityProperties,
                         Rgba, RigidTransform, RotationMatrix, Simulator,
                         SpatialInertia, Sphere)

In [ ]:
meshcat = Meshcat()

In [ ]:
class DesiredStateSource(LeafSystem):
    def __init__(self, speed, num_actuators):
        super().__init__()
        self._speed = speed
        self._num_actuators = num_actuators

        self.DeclareVectorOutputPort(
            "desired_state",
            BasicVector(num_actuators * 2),
            self.CalcOutput
        )

    def CalcOutput(self, context, output):
        t = context.get_time()
        value = output.get_mutable_value()

        value[:self._num_actuators] = self._speed * t
        value[self._num_actuators:] = self._speed

In [ ]:
youngs_modulus = 0.18e8
shear_modulus = 0.06e8 + 1
mass_density = 1000
diameter = 0.003
P_gain = 1e6
speed = 0.8

In [23]:
builder = DiagramBuilder()

plant_config = MultibodyPlantConfig()
plant_config.time_step = 1e-4

plant, scene_graph = AddMultibodyPlant(plant_config, builder)
deformable_model = plant.mutable_deformable_model()

nodes = np.loadtxt("./models/n4.txt").T
rope = Filament(False, nodes, Filament.CircularCrossSection(diameter=3e-3))
geometry = GeometryInstance(RigidTransform(), rope, "rope")

illus_props = IllustrationProperties()
illus_props.AddProperty("phong", "diffuse", Rgba(0.7,0.5,0.4,1))
geometry.set_illustration_properties(illus_props)

proximity_props = ProximityProperties()
AddContactMaterial(
    properties=proximity_props,
    friction=CoulombFriction(0,0),
    dissipation=0.01,
    point_stiffness=1e5,
)
geometry.set_proximity_properties(proximity_props)

config = DeformableBodyConfig()
config.set_youngs_modulus(youngs_modulus)
config.set_poissons_ratio(0.5 * youngs_modulus / shear_modulus - 1)
config.set_mass_density(mass_density)
config.set_mass_damping_coefficient(1.0)

unused_resolution_hint = 9999
rope_body_id = deformable_model.RegisterDeformableBody(
    geometry, config, unused_resolution_hint)

# Add link 1
sphere_radius = diameter * 2
link1 = plant.AddRigidBody("link1", SpatialInertia.SolidSphereWithDensity(100, sphere_radius))
X_WL1 = RigidTransform(
    RotationMatrix.MakeFromOneVector(nodes[:,0] - nodes[:,-1], 2),
    nodes[:,0]
)
joint1 = plant.AddJoint(PrismaticJoint(
    "joint1",
    plant.AddFrame(FixedOffsetFrame("world_offset_frame1", plant.world_frame(), X_WL1)),
    link1.body_frame(),
    [0,0,1]
))
actuator1 = plant.AddJointActuator("actuator1", joint1)
actuator1.set_controller_gains(PdControllerGains(p=P_gain, d=0.0))
plant.RegisterVisualGeometry(
    link1, RigidTransform(), Sphere(sphere_radius), "link1", np.array([1, 0, 0, 1])
)

# Add link 2
link2 = plant.AddRigidBody("link2", SpatialInertia.SolidSphereWithDensity(100, sphere_radius))
X_WL2 = RigidTransform(
    RotationMatrix.MakeFromOneVector(nodes[:,-1] - nodes[:,0], 2),
    nodes[:,-1]
)
joint2 = plant.AddJoint(PrismaticJoint(
    "joint2",
    plant.AddFrame(FixedOffsetFrame("world_offset_frame2", plant.world_frame(), X_WL2)),
    link2.body_frame(),
    [0,0,1]
))
actuator2 = plant.AddJointActuator("actuator2", joint2)
actuator2.set_controller_gains(PdControllerGains(p=P_gain, d=0.0))
plant.RegisterVisualGeometry(
    link2, RigidTransform(), Sphere(sphere_radius), "link2", np.array([1, 0, 0, 1])
)

# Attach filament
deformable_model.AddFixedConstraint(
    rope_body_id, link1, X_WL1.inverse(), Sphere(1e-6), RigidTransform()
)
deformable_model.AddFixedConstraint(
    rope_body_id, link2, X_WL2.inverse(), Sphere(1e-6), RigidTransform()
)

plant.mutable_gravity_field().set_gravity_vector(np.zeros(3))
plant.Finalize()

# Add position source
source = builder.AddSystem(DesiredStateSource(speed / 2, 2))
builder.Connect(source.get_output_port(), plant.get_desired_state_input_port(ModelInstanceIndex(1)))

# Add Meshcat visualizer
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams(publish_period=1/500)
)

diagram = builder.Build()

simulator = Simulator(diagram)

visualizer.StartRecording()
simulator.AdvanceTo(0.7)
visualizer.StopRecording()
visualizer.PublishRecording()